# File 1:Firms' Preprocessing

Working on firm level data <br>
- This will generate patent_firm_full_clean.pkl file

In [53]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

In [54]:
# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [55]:
import pandas as pd
import nltk
import numpy as np
import requests
import pickle

In [56]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [57]:
patents_new=pd.read_csv('hassan_all_en_new.csv')

In [58]:
patents_new.head(2)

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn_date,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr
0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,WO9303532A1,...,2002-07-11,WO0199222A3,WO0119827W,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...
1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,US5121044A,...,1993-05-04\n1994-03-09\n1994-09-20\n1995-07-18...,US5208526A\nEP0557287A4\nUS5349535A\nUS5434020...,US63645090A\nEP91915464A\nUS96399792A\nUS15296...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...


- appln                     The identifier of the application (with letter at the end, seems Google omits that)
- appln_id                  The (internal) patstat identifier of the application
- family_id                 The (internal) patstat identifier of the family that the application is a member of sets  - sets                      Indicate which sets (fuel-cell = 1, electric  vehicle = 2, both = 3) was matched
- direct                    Indicate whether this application was found directly (title/abstract/cpc/ipc) or      indirectly (family association)
- granted                   Whether or not the application was granted
- earliest_appln            Per family, the earliest application
- earliest_filing_date      The date of the family's earliest application
- publn                     Identifiers of publications associated with this application (*)
- publn_date                The dates of associated publications (*)
- publn_first_grant         Indicates for each publication if its the first one with granted status
- num_cpc_class             The number of CPC classes for this application
- cpc_class                 The CPC class symbols for this application
- num_ipc_class             The number of IPC classes for this application
- ipc_class                 The IPC class symbols for this application
- num_cited_publn           The number of forward citations (this applications refers to other publications)
- cited_publn_date          The dates of cited publications (**)
- cited_publn               The identifiers of cited publications (**)
- cited_appln               The identifiers of the applications  associated with cited publications (**)
- cited_appln_id            The (internal) patstat identifiers of the  applications associated with cited publications (**)
- cited_family_id           The (internal) patstat identifiers of the  family associated with cited publications (**)
- num_cited_family          The total number of distinct families  associated with cited publications (***)
- num_citing_publn          The number of backward citations (other  publications refer to this application)
- citing_publn_date         The dates of citing publications (**)
- citing_publn              The identifiers of citing publications (**)
- citing_appln              The identifiers of the applications  associated with citing publications (**)
- citing_appln_id           The (internal) patstat identifiers of the  applications associated with citing publications (**)
- citing_family_id          The (internal) patstat identifiers of the  family associated with citing publications (**)
- num_citing_family         The total number of distinct families  associated with citing publications (***)
- earliest_applt            The name of the (first) applicant of the  earliest publication of this application 
- current_applt             The name of the (first) applicant of the most recent publication of this application
- appln_title               The title of this application

>>
>> (*/**) These are one per line, in consistent order


>> (***) These seem to be the numbers that google uses to count citations. You can get this from the export if you aggregate all applications within a family and discard duplicates.


In [59]:
patents_new.shape

(43432, 34)

In [60]:
patents_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43432 entries, 0 to 43431
Data columns (total 34 columns):
appln                   43432 non-null object
appln_id                43432 non-null int64
family_id               43432 non-null int64
sets                    43432 non-null int64
direct                  43432 non-null int64
granted                 43432 non-null int64
earliest_appln          43432 non-null object
earliest_filing_date    43432 non-null object
publn_date              43432 non-null object
publn                   43432 non-null object
publn_first_grant       43432 non-null object
publn_claims            43432 non-null object
num_cpc_class           43432 non-null int64
cpc_class               39286 non-null object
num_ipc_class           43432 non-null int64
ipc_class               43399 non-null object
num_cited_publn         43432 non-null int64
cited_publn_date        29098 non-null object
cited_publn             29098 non-null object
cited_appln             2

### Working with firm variable

i.e earliest_applt. The firms that created knowldge and innovation

In [61]:
#patents_new[patents_new.current_applt=='TOYOTA MOTOR CORP']

In [62]:
#ri[ri.violation == 'Speeding'].driver_gender.value_counts()

In [63]:
assignee=patents_new['earliest_applt']

In [64]:
type(assignee)

pandas.core.series.Series

In [65]:
# remove punctuation
assignee = patents_new['earliest_applt'].str.replace('[^\w\s]','')
assignee = patents_new['earliest_applt'].replace(".",'') 
assignee = patents_new['earliest_applt'].str.replace('\[.*?\]','')
assignee = patents_new['earliest_applt'].str.replace(",",'' )

In [66]:
patents_new['assignee']=assignee

In [67]:
assignee[9]

'STEYR-DAIMLER-PUCH AG WIEN AT'

In [68]:
assignee[7]

'FRONIUS SCHWEISMASCHINEN PRODUKTION GMBH & CO. KG PETTENBACH'

### Round2

In [69]:
assignee=patents_new['assignee'].str.split()
assignee=patents_new['assignee'].str.lower()

In [70]:
patents_new['assignee']=assignee

In [71]:
patents_new.assignee[7]

'fronius schweismaschinen produktion gmbh & co. kg pettenbach'

In [72]:
assignee[7]

'fronius schweismaschinen produktion gmbh & co. kg pettenbach'

In [73]:
assignee[9]

'steyr-daimler-puch ag wien at'

In [74]:
patents_new.assignee[9]

'steyr-daimler-puch ag wien at'

In [75]:
patents_new.assignee.value_counts()[:10]

toyota jidosha kabushiki kaisha       2562
toyota motor corp                     2301
hyundai motor company                 1990
honda motor co ltd                    1206
ford global technologies llc          1009
nissan motor co ltd                    996
nissan motor co. ltd.                  908
honda motor co. ltd.                   814
robert bosch gmbh                      557
honda giken kogyo kabushiki kaisha     491
Name: assignee, dtype: int64

In [76]:
patents_new['assignee'] = patents_new['assignee'].str.replace("[^a-zA-Z#]"," ") 
patents_new['assignee'].value_counts()[:10]

toyota jidosha kabushiki kaisha       2577
toyota motor corp                     2301
hyundai motor company                 1990
honda motor co ltd                    1206
ford global technologies llc          1009
nissan motor co ltd                    996
nissan motor co  ltd                   908
honda motor co  ltd                    814
robert bosch gmbh                      557
honda giken kogyo kabushiki kaisha     492
Name: assignee, dtype: int64

In [77]:
patents_new['assignee'][7]

'fronius schweismaschinen produktion gmbh   co  kg pettenbach'

### Getting value count

In [78]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:10]

,unique_values,counts
0,toyota jidosha kabushiki kaisha,2577
1,toyota motor corp,2301
2,hyundai motor company,1990
3,honda motor co ltd,1206
4,ford global technologies llc,1009
5,nissan motor co ltd,996
6,nissan motor co ltd,908
7,honda motor co ltd,814
8,robert bosch gmbh,557
9,honda giken kogyo kabushiki kaisha,492


### to chekc specifi automobile company

In [79]:
df_val_counts[df_val_counts.unique_values.str.contains('toyota',regex=True)][:8]

,unique_values,counts
0,toyota jidosha kabushiki kaisha,2577
1,toyota motor corp,2301
10,toyota motor corporation,376
15,toyota motor co ltd,228
87,toyota motor co ltd,52
120,toyota industries corp,38
151,toyota jidosha kabushiki kaisya,28
176,kabushiki kaisha toyota jidoshokki,24


In [80]:
len(df_val_counts[df_val_counts.unique_values.str.contains('toyota',regex=True)])

42

So word "toyota" has been used 42 time and we know exact rows and exact title where it has been used.

# cleaning data

In [81]:
df_val_counts[:30]

,unique_values,counts
0,toyota jidosha kabushiki kaisha,2577
1,toyota motor corp,2301
2,hyundai motor company,1990
3,honda motor co ltd,1206
4,ford global technologies llc,1009
5,nissan motor co ltd,996
6,nissan motor co ltd,908
7,honda motor co ltd,814
8,robert bosch gmbh,557
9,honda giken kogyo kabushiki kaisha,492


We need no repeated company name

### Solving toyota duplication

In [83]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota jidosha kabushiki kaisha','toyota motor corp')

In [84]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor corporation','toyota motor corp')

In [85]:
# in three steps, I ll again check if our str is working

value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data2 = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data2)
df_val_counts[df_val_counts.unique_values.str.contains('toyota',regex=True)][:10]

,unique_values,counts
0,toyota motor corp,5254
13,toyota motor co ltd,228
85,toyota motor co ltd,52
121,toyota industries corp,38
149,toyota jidosha kabushiki kaisya,28
175,kabushiki kaisha toyota jidoshokki,24
203,toyota motor engineering manufacturing north...,20
220,toyota jidosha k k toyota,19
239,toyota jidosha kk,17
492,toyota autom loom works ltd,8


In [92]:
df_val_counts.loc[13]

unique_values    toyota motor co  ltd 
counts                             228
Name: 13, dtype: object

In [93]:
df_val_counts.loc[85]

unique_values    toyota motor co ltd
counts                            52
Name: 85, dtype: object

In [96]:
# for full name of company
df_val_counts.unique_values[203]

'toyota motor engineering   manufacturing north america inc '

In [97]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor co  ltd ','toyota motor corp')

In [98]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor co ltd','toyota motor corp')

In [99]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota industries corp','toyota motor corp')

In [100]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota jidosha kabushiki kaisya','toyota motor corp')

In [101]:
patents_new['assignee']=patents_new['assignee'].str.replace('kabushiki kaisha toyota jidoshokki','toyota motor corp')

In [102]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor engineering   manufacturing north america inc ','toyota motor corp')

In [103]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota jidosha k k toyota','toyota motor corp')

In [104]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data2 = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data2)
df_val_counts[df_val_counts.unique_values.str.contains('toyota',regex=True)][:10]

,unique_values,counts
0,toyota motor corp,5644
211,toyota jidosha k k toyota,19
229,toyota jidosha kk,17
498,toyota autom loom works ltd,8
560,toyota motor corp toyota,7
566,toyota jidosha k k,7
710,toyota central res dev lab inc,6
897,toyota central r d labs inc,5
1048,toyota motor corp,4
1830,toyota jidoshokki kk,2


In [105]:
# for exact string catch
df_val_counts.unique_values[211]

'toyota jidosha k k  toyota'

In [106]:
df_val_counts.unique_values[229]

'toyota jidosha kk'

In [107]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota jidosha k k  toyota','toyota motor corp')

In [108]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota jidosha kk','toyota motor corp')

In [109]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data2 = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data2)
df_val_counts[df_val_counts.unique_values.str.contains('toyota',regex=True)][:10]

,unique_values,counts
0,toyota motor corp,5680
500,toyota autom loom works ltd,8
529,toyota jidosha k k,7
576,toyota motor corp toyota,7
704,toyota central res dev lab inc,6
871,toyota central r d labs inc,5
1125,toyota motor corp,4
1984,toyota jidoshokki kk,2
2383,toyota shatai kabushiki kaisha,2
2463,toyota boshoku corp,2


We will stop here with toyota and we shall move next to other entities

In [110]:
patents_new['assignee'].value_counts()[:20]

toyota motor corp                            5680
hyundai motor company                        1990
honda motor co ltd                           1206
ford global technologies llc                 1009
nissan motor co ltd                           996
nissan motor co  ltd                          908
honda motor co  ltd                           814
robert bosch gmbh                             557
honda giken kogyo kabushiki kaisha            492
denso corp                                    262
mitsubishi motors corp                        260
peugeot citroen automobiles sa                244
hitachi ltd                                   229
lsis co  ltd                                  222
gm global technology operations llc           212
hyundai motor co ltd                          200
mazda motor corp                              194
sony corporation                              192
aisin aw co  ltd                              187
mitsubishi jidosha kogyo kabushiki kaisha     186


### save your work uptill Toyota

In [111]:
import pickle
patents_new.to_pickle('patent_firm_full_clean.pkl')

In [112]:
patents_new.to_csv('patent_firm_full_clean.csv')

### For honda

In [113]:
df_val_counts[df_val_counts.unique_values.str.contains('honda',regex=True)]

,unique_values,counts
2,honda motor co ltd,1206
6,honda motor co ltd,814
8,honda giken kogyo kabushiki kaisha,492
168,honda giken kogyo kabusiki kaisha,25
190,honda giken kogyo k k,21
191,honda giken kogyo kabushiki kaisha also tradi...,21
195,honda giken kogyo k k,20
243,khonda motor ko ltd,16
245,honda giken kogyo k k tokio tokyo,16
290,honda giken kogyo kk,13


In [114]:
len(df_val_counts[df_val_counts.unique_values.str.contains('honda',regex=True)])

27

In [115]:
# for exact string catch
df_val_counts.unique_values[2]

'honda motor co ltd'

In [116]:
df_val_counts.unique_values[6]

'honda motor co  ltd '

In [118]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co  ltd' ,'honda motor co ltd')

In [117]:
print(df_val_counts.unique_values[8])
print(df_val_counts.unique_values[168])

honda giken kogyo kabushiki kaisha
honda giken kogyo kabusiki kaisha


In [119]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda giken kogyo kabushiki kaisha','honda motor co ltd')

In [120]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda giken kogyo kabusiki kaisha','honda motor co ltd')

In [121]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data2 = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data2)
df_val_counts[df_val_counts.unique_values.str.contains('honda',regex=True)]

,unique_values,counts
2,honda motor co ltd,1734
6,honda motor co ltd,819
186,honda motor co ltd also trading as honda moto...,21
189,honda giken kogyo k k,21
191,honda giken kogyo k k,20
233,honda giken kogyo k k tokio tokyo,16
241,khonda motor ko ltd,16
283,honda giken kogyo kk,13
913,honda motor company limited,4
1825,honda giken kogyo k k tokio tokyo jp,2


In [122]:
print(df_val_counts.unique_values[2])
print(df_val_counts.unique_values[6])

honda motor co ltd
honda motor co ltd 


In [123]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co ltd','honda motor company ltd')

In [124]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co ltd','honda motor company ltd')

In [125]:
print(df_val_counts.unique_values[186])
print(df_val_counts.unique_values[191])
print(df_val_counts.unique_values[233])
print(df_val_counts.unique_values[241])
print(df_val_counts.unique_values[283])

honda motor co ltd  also trading as honda motor co ltd    
honda giken kogyo k  k 
honda giken kogyo k k  tokio tokyo
khonda motor ko  ltd 
honda giken kogyo kk


In [126]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co ltd  also trading as honda motor co ltd','honda motor company ltd')

In [127]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda giken kogyo k  k','honda motor company ltd')

In [128]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda giken kogyo k k  tokio tokyo','honda motor company ltd')

In [129]:
patents_new['assignee']=patents_new['assignee'].str.replace('khonda motor ko  ltd','honda motor company ltd')

In [130]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda giken kogyo kk','honda motor company ltd')

In [131]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data2 = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data2)
df_val_counts[df_val_counts.unique_values.str.contains('honda',regex=True)]

,unique_values,counts
2,honda motor company ltd,1764
6,honda motor company ltd,856
186,honda motor company ltd also trading as honda...,21
190,honda giken kogyo k k,21
1057,honda motor company limited,4
2005,honda motor company ltd jp,2
3244,honda giken k k,1
3262,honda giken kogyo kabushiki kaish,1
4223,honda giken kogyo kasbushiki kaisha,1
5308,honda giken kogyo,1


In [132]:
print(df_val_counts.unique_values[2])
print(df_val_counts.unique_values[6])

honda motor company ltd
honda motor company ltd 


In [133]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor company ltd','honda motor ltd')

In [134]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co ltd  also trading as honda motor co ltd','honda motor company ltd')

In [135]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data2 = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data2)
df_val_counts[df_val_counts.unique_values.str.contains('honda',regex=True)]

,unique_values,counts
2,honda motor ltd,1764
6,honda motor ltd,856
185,honda motor ltd also trading as honda motor l...,21
186,honda giken kogyo k k,21
1107,honda motor company limited,4
3157,honda motor ltd jp,2
3232,honda giken k k,1
3611,honda giken kogyo kabushiki kaish,1
4224,honda giken kogyo kasbushiki kaisha,1
5308,honda giken kogyo,1


In [142]:
df_val_counts.unique_values[6]

'honda motor ltd '

In [143]:
df_val_counts.unique_values[185]

'honda motor ltd  also trading as honda motor ltd    '

In [141]:
df_val_counts.unique_values[186]

'honda giken kogyo k k '

In [138]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor ltd ','honda motor ltd')

In [144]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor ltd  also trading as honda motor ltd    ','honda motor ltd')

In [145]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda giken kogyo k k ','honda motor ltd')

In [146]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data2 = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data2)
df_val_counts[df_val_counts.unique_values.str.contains('honda',regex=True)]

,unique_values,counts
1,honda motor ltd,2641
187,honda motor ltd also trading as honda motor lt...,21
1016,honda motor company limited,4
2583,honda motor ltdjp,2
3626,honda giken kogyo kabushiki kaish,1
4291,honda giken kogyo,1
5013,honda giken k k,1
5206,honda giken kogyo kasbushiki kaisha,1
5796,honda motor co ltd,1
6050,honda giken kogyo k k,1


In [147]:
df_val_counts.unique_values[187]

'honda motor ltd also trading as honda motor ltd   '

In [148]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor ltd also trading as honda motor ltd   ','honda motor ltd')

In [149]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data2 = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data2)
df_val_counts[df_val_counts.unique_values.str.contains('honda',regex=True)]

,unique_values,counts
1,honda motor ltd,2662
1116,honda motor company limited,4
2511,honda motor ltdjp,2
3808,honda giken kogyo kabushiki kaish,1
4190,honda giken kogyo,1
4460,honda giken kogyo kasbushiki kaisha,1
4963,honda giken k k,1
5795,honda motor co ltd,1
6049,honda giken kogyo k k,1
6074,honda tomoyuki,1


In [157]:
patents_new['assignee'].value_counts()[:40]

toyota motor corp                                     5680
honda motor ltd                                       2662
hyundai motor company                                 1990
nissan motor co ltd                                   1904
ford global technologies llc                          1009
robert bosch gmbh                                      557
denso corp                                             262
mitsubishi motors corp                                 260
peugeot citroen automobiles sa                         244
hitachi ltd                                            229
lsis co  ltd                                           222
gm global technology operations llc                    212
hyundai motor co ltd                                   200
mazda motor corp                                       194
sony corporation                                       192
aisin aw co  ltd                                       187
mitsubishi jidosha kogyo kabushiki kaisha              1

### save work

In [151]:
patents_new.to_csv('patent_firm_full_clean.csv')
patents_new.to_pickle('patent_firm_full_clean.pkl')

# Work with repeating firms in top 40

### nissan

In [153]:
df_val_counts[df_val_counts.unique_values.str.contains('nissan',regex=True)]

,unique_values,counts
4,nissan motor co ltd,996
5,nissan motor co ltd,908
67,nissan motor co ltd,64
74,nissan motor,58
113,nissan motor ko ltd,39
117,nissan motor company limited,37
170,nissan diesel motor co ltd,23
174,nissan diesel motor co ltd,23
193,nissan motor co ltd,20
409,zhengzhou nissan automobile co ltd,9


In [154]:
df_val_counts.unique_values[4]

'nissan motor co ltd'

In [155]:
df_val_counts.unique_values[5]

'nissan motor co  ltd '

In [156]:
patents_new['assignee']=patents_new['assignee'].str.replace('nissan motor co  ltd ','nissan motor co ltd')

### suzuki

In [188]:
df_val_counts[df_val_counts.unique_values.str.contains('suzuki',regex=True)]

,unique_values,counts
19,suzuki motor corporation,182
21,suzuki motor corp,158
204,suzuki motor corp,19
308,suzuki motor corp hamamatsu,11
622,suzuki naoto,6
1345,suzuki masahiro,3
2170,suzuki hiroyuki,2
2948,suzuki minoru,2
3265,suzuki takashi,1
3424,suzuki co ltd,1


In [187]:
df_val_counts.unique_values[19]

'suzuki motor corporation'

In [183]:
df_val_counts.unique_values[21]

'suzuki motor corp'

In [175]:
df_val_counts.unique_values[204]

'suzuki motor corp '

In [176]:
df_val_counts.unique_values[308]

'suzuki motor corp  hamamatsu'

In [184]:
patents_new['assignee']=patents_new['assignee'].str.replace('suzuki motor corporation' , 'suzuki motor corp')

In [190]:
patents_new['assignee']=patents_new['assignee'].str.replace('suzuki motor corp ' , 'suzuki motor corp')

In [192]:
patents_new['assignee']=patents_new['assignee'].str.replace('suzuki motor corp  hamamatsu' , 'suzuki motor corp')

In [193]:
patents_new['assignee'].value_counts()[:40]

toyota motor corp                                     5680
honda motor ltd                                       2662
hyundai motor company                                 1990
nissan motor co ltd                                   1904
ford global technologies llc                          1009
robert bosch gmbh                                      557
suzuki motor corp                                      359
denso corp                                             262
mitsubishi motors corp                                 260
peugeot citroen automobiles sa                         244
hitachi ltd                                            229
lsis co  ltd                                           222
gm global technology operations llc                    212
hyundai motor co ltd                                   200
mazda motor corp                                       194
sony corporation                                       192
aisin aw co  ltd                                       1

### ford

In [200]:
df_val_counts[df_val_counts.unique_values.str.contains('ford',regex=True)]

,unique_values,counts
3,ford global technologies llc,1009
31,ford global tech llc,123
45,ford global technologies inc,88
54,ford motor company,76
160,ford motor co,25
180,ford global technol inc,22
210,ford gloubal teknolodzhiz elelsi,18
328,ford global technologies llc,11
339,ford global technolgies llc,10
477,ford global tekhnolodzhis llk,8


In [196]:
df_val_counts.unique_values[3]

'ford global technologies llc'

In [197]:
df_val_counts.unique_values[31]

'ford global tech llc'

In [195]:
df_val_counts.unique_values[45]

'ford global technologies inc '

In [202]:
df_val_counts.unique_values[54]

'ford motor company'

In [198]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global tech llc' , 'ford global technologies llc')

In [199]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies inc ' , 'ford global technologies llc')

In [203]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford motor company' , 'ford global technologies llc')

In [204]:
patents_new['assignee'].value_counts()[:40]

toyota motor corp                                     5680
honda motor ltd                                       2662
hyundai motor company                                 1990
nissan motor co ltd                                   1904
ford global technologies llc                          1296
robert bosch gmbh                                      557
suzuki motor corp                                      359
denso corp                                             262
mitsubishi motors corp                                 260
peugeot citroen automobiles sa                         244
hitachi ltd                                            229
lsis co  ltd                                           222
gm global technology operations llc                    212
hyundai motor co ltd                                   200
mazda motor corp                                       194
sony corporation                                       192
aisin aw co  ltd                                       1

### general motors corporation

In [205]:
df_val_counts.unique_values[:40]

0                                     toyota motor corp
1                                       honda motor ltd
2                                 hyundai motor company
3                          ford global technologies llc
4                                   nissan motor co ltd
5                                 nissan motor co  ltd 
6                                     robert bosch gmbh
7                                            denso corp
8                                mitsubishi motors corp
9                        peugeot citroen automobiles sa
10                                         hitachi ltd 
11                                        lsis co  ltd 
12                  gm global technology operations llc
13                                 hyundai motor co ltd
14                                     mazda motor corp
15                                     sony corporation
16                                    aisin aw co  ltd 
17            mitsubishi jidosha kogyo kabushiki

In [206]:
df_val_counts.unique_values[12]

'gm global technology operations llc'

In [207]:
df_val_counts.unique_values[25]

'general motors corporation'

In [208]:
patents_new['assignee']=patents_new['assignee'].str.replace('gm global technology operations llc' , 'general motors corporation')

In [211]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:40]

,unique_values,counts
0,toyota motor corp,5680
1,honda motor ltd,2662
2,hyundai motor company,1990
3,nissan motor co ltd,1904
4,ford global technologies llc,1296
5,robert bosch gmbh,557
6,suzuki motor corp,359
7,general motors corporation,346
8,denso corp,262
9,mitsubishi motors corp,260


### mitsubishi motors corp

In [212]:
df_val_counts[df_val_counts.unique_values.str.contains('mitsubishi',regex=True)]

,unique_values,counts
9,mitsubishi motors corp,260
17,mitsubishi jidosha kogyo kabushiki kaisha,186
25,mitsubishi electric corporation,127
29,mitsubishi electric corp,117
71,mitsubishi heavy ind ltd,57
75,mitsubishi heavy industries ltd,53
83,mitsubishi fuso truck bus corp,47
137,mitsubishi denki kabushiki kaisha,28
201,mitsubishi fuso truck and bus corporation,18
211,mitsubishi motors corp,17


In [213]:
df_val_counts.unique_values[9]

'mitsubishi motors corp'

In [214]:
df_val_counts.unique_values[17]

'mitsubishi jidosha kogyo kabushiki kaisha'

In [215]:
df_val_counts.unique_values[25]

'mitsubishi electric corporation'

In [216]:
df_val_counts.unique_values[29]

'mitsubishi electric corp'

In [217]:
df_val_counts.unique_values[71]

'mitsubishi heavy ind ltd'

In [218]:
df_val_counts.unique_values[75]

'mitsubishi heavy industries ltd '

In [219]:
df_val_counts.unique_values[83]

'mitsubishi fuso truck   bus corp'

In [220]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi jidosha kogyo kabushiki kaisha' , 'mitsubishi motors corp')

In [221]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi electric corporation' , 'mitsubishi motors corp')

In [222]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi electric corp' , 'mitsubishi motors corp')

In [223]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi heavy ind ltd' , 'mitsubishi motors corp')

In [224]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi heavy industries ltd '  , 'mitsubishi motors corp')

In [225]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi fuso truck   bus corp'  , 'mitsubishi motors corp')

In [230]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:40]

,unique_values,counts
0,toyota motor corp,5680
1,honda motor ltd,2662
2,hyundai motor company,2190
3,nissan motor co ltd,1904
4,ford global technologies llc,1296
5,mitsubishi motors corp,847
6,robert bosch gmbh,557
7,suzuki motor corp,359
8,general motors corporation,346
9,denso corp,262


### hyundai motor company vs hyundai motor co ltd

In [227]:
df_val_counts.unique_values[13]

'hyundai motor co ltd'

In [228]:
df_val_counts.unique_values[2]

'hyundai motor company'

In [229]:
patents_new['assignee']=patents_new['assignee'].str.replace('hyundai motor co ltd'  , 'hyundai motor company')

### save work

In [231]:
patents_new.to_csv('patent_firm_full_clean.csv')
patents_new.to_pickle('patent_firm_full_clean.pkl')

### hitachi ltd

In [233]:
df_val_counts[df_val_counts.unique_values.str.contains('hitachi',regex=True)][:5]

,unique_values,counts
11,hitachi ltd,229
18,hitachi ltd,155
62,hitachi automotive systems ltd,59
104,hitachi automotive systems ltd,38
298,hitachi ltd tokio tokyo jp,11


In [234]:
df_val_counts.unique_values[11]

'hitachi ltd '

In [235]:
df_val_counts.unique_values[18]

'hitachi ltd'

In [236]:
df_val_counts.unique_values[62]

'hitachi automotive systems ltd '

In [237]:
df_val_counts.unique_values[104]

'hitachi automotive systems ltd'

In [238]:
patents_new['assignee']=patents_new['assignee'].str.replace('hitachi ltd '  , 'hitachi automotive systems ltd')

In [239]:
patents_new['assignee']=patents_new['assignee'].str.replace('hitachi ltd'  , 'hitachi automotive systems ltd')

In [240]:
patents_new['assignee']=patents_new['assignee'].str.replace('hitachi automotive systems ltd '  , 'hitachi automotive systems ltd')

In [241]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:40]

,unique_values,counts
0,toyota motor corp,5680
1,honda motor ltd,2662
2,hyundai motor company,2190
3,nissan motor co ltd,1904
4,ford global technologies llc,1296
5,mitsubishi motors corp,847
6,robert bosch gmbh,557
7,hitachi automotive systems ltd,481
8,suzuki motor corp,359
9,general motors corporation,346


### daimler

In [242]:
df_val_counts[df_val_counts.unique_values.str.contains('daimler',regex=True)][:5]

,unique_values,counts
31,daimler ag,99
34,daimlerchrysler ag,93
179,daimlerchrysler corporation,20
368,daimler benz aktiengesellschaft,10
458,daimler benz ag,8


In [243]:
df_val_counts.unique_values[31]

'daimler ag'

In [244]:
df_val_counts.unique_values[34]

'daimlerchrysler ag'

In [245]:
df_val_counts.unique_values[179]

'daimlerchrysler corporation'

In [246]:
patents_new['assignee']=patents_new['assignee'].str.replace('daimlerchrysler ag'  , 'daimler ag')

In [247]:
patents_new['assignee']=patents_new['assignee'].str.replace('daimlerchrysler corporation'  , 'daimler ag')

In [248]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:40]

,unique_values,counts
0,toyota motor corp,5680
1,honda motor ltd,2662
2,hyundai motor company,2190
3,nissan motor co ltd,1904
4,ford global technologies llc,1296
5,mitsubishi motors corp,847
6,robert bosch gmbh,557
7,hitachi automotive systems ltd,481
8,suzuki motor corp,359
9,general motors corporation,346


### aisin

In [250]:
df_val_counts[df_val_counts.unique_values.str.contains('aisin',regex=True)][:5]

,unique_values,counts
16,aisin aw co ltd,187
27,aisin aw co ltd,110
58,aisin seiki co ltd,62
77,aisin seiki kabushiki kaisha,46
163,aisin ai co ltd,22


In [251]:
df_val_counts.unique_values[16]

'aisin aw co  ltd '

In [252]:
df_val_counts.unique_values[27]

'aisin aw co ltd'

In [253]:
df_val_counts.unique_values[58]

'aisin seiki co ltd'

In [254]:
df_val_counts.unique_values[77]

'aisin seiki kabushiki kaisha'

In [255]:
patents_new['assignee']=patents_new['assignee'].str.replace('aisin aw co  ltd '  , 'aisin seiki co ltd')

In [256]:
patents_new['assignee']=patents_new['assignee'].str.replace('aisin aw co ltd'  , 'aisin seiki co ltd')

In [257]:
patents_new['assignee']=patents_new['assignee'].str.replace('aisin seiki kabushiki kaisha'  , 'aisin seiki co ltd')

In [258]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:40]

,unique_values,counts
0,toyota motor corp,5680
1,honda motor ltd,2662
2,hyundai motor company,2190
3,nissan motor co ltd,1904
4,ford global technologies llc,1296
5,mitsubishi motors corp,847
6,robert bosch gmbh,557
7,hitachi automotive systems ltd,481
8,aisin seiki co ltd,405
9,suzuki motor corp,359


### save work

In [259]:
patents_new.to_csv('patent_firm_full_clean.csv')
patents_new.to_pickle('patent_firm_full_clean.pkl')

### denso

In [269]:
df_val_counts[df_val_counts.unique_values.str.contains('denso',regex=True)][:5]

,unique_values,counts
11,denso corporation,262
25,denso corporationoration,118
266,nippondenso co ltd,12
413,nippondenso co ltd,9
426,sumitomo denso kk,8


In [266]:
df_val_counts.unique_values[11]

'denso corporation'

In [267]:
df_val_counts.unique_values[25]

'denso corporationoration'

In [268]:
patents_new['assignee']=patents_new['assignee'].str.replace('denso corporationoration' , 'denso corporation')

In [270]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:40]

,unique_values,counts
0,toyota motor corp,5680
1,honda motor ltd,2662
2,hyundai motor company,2190
3,nissan motor co ltd,1904
4,ford global technologies llc,1296
5,mitsubishi motors corp,847
6,robert bosch gmbh,557
7,hitachi automotive systems ltd,481
8,aisin seiki co ltd,405
9,denso corporation,380


### panasonic

In [272]:
df_val_counts[df_val_counts.unique_values.str.contains('panasonic',regex=True)][:5]

,unique_values,counts
25,panasonic corporation,114
35,panasonic corp,84
77,panasonic intellectual property management co ...,44
117,panasonic electric works co ltd,30
188,panasonic ev energy co ltd,18


In [273]:
df_val_counts.unique_values[25]

'panasonic corporation'

In [274]:
df_val_counts.unique_values[35]

'panasonic corp'

In [277]:
patents_new['assignee']=patents_new['assignee'].str.replace('panasonic corporationoration' , 'panasonic corporation')

In [278]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:40]

,unique_values,counts
0,toyota motor corp,5680
1,honda motor ltd,2662
2,hyundai motor company,2190
3,nissan motor co ltd,1904
4,ford global technologies llc,1296
5,mitsubishi motors corp,847
6,robert bosch gmbh,557
7,hitachi automotive systems ltd,481
8,aisin seiki co ltd,405
9,denso corporation,380


### renault 

In [279]:
df_val_counts[df_val_counts.unique_values.str.contains('renault',regex=True)][:5]

,unique_values,counts
20,renault s a s,148
37,renault s a s,78
55,renault sas,61
73,renault,45
171,renault sas societe par actions simplifiee,20


In [280]:
df_val_counts.unique_values[20]

'renault s a s '

In [281]:
df_val_counts.unique_values[37]

'renault s a s'

In [282]:
df_val_counts.unique_values[55]

'renault sas'

In [283]:
patents_new['assignee']=patents_new['assignee'].str.replace('renault s a s ' , 'renault sas')

In [284]:
patents_new['assignee']=patents_new['assignee'].str.replace('renault s a s' , 'renault sas')

In [285]:
value_counts = patents_new['assignee'].value_counts(dropna=True, sort=True)
data = value_counts.rename_axis('unique_values').reset_index(name='counts')
df_val_counts = pd.DataFrame(data)
df_val_counts[:40]

,unique_values,counts
0,toyota motor corp,5680
1,honda motor ltd,2662
2,hyundai motor company,2190
3,nissan motor co ltd,1904
4,ford global technologies llc,1296
5,mitsubishi motors corp,847
6,robert bosch gmbh,557
7,hitachi automotive systems ltd,481
8,aisin seiki co ltd,405
9,denso corporation,380


### save work

In [286]:
patents_new.to_csv('patent_firm_full_clean.csv')
patents_new.to_pickle('patent_firm_full_clean.pkl')

### complete work